In [ ]:
#pip install webdriver-manager
#pip install selenium

import pandas as pd
import requests
import os
from time import sleep


# Atualização do webdriver automático - > https://www.youtube.com/watch?v=2EIfXi2-vCI

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

def unique(list1):
    list_set = set(list1)
    unique_list = (list(list_set))
    return unique_list

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = servico)
driver.get('https://www.mercante.transportes.gov.br/g36127/servlet/serpro.siscomex.mercante.servlet.MercanteController')

# Excel antigo pode ser xls

try:
    df = pd.read_excel(r'C:\Users\sabedot\Desktop\automacao\GUARITA.xlsx')

except:
    df = pd.read_excel(r'C:\Users\sabedot\Desktop\automacao\GUARITA.xls')
    
ncm = []
for item in df['NCM'].astype('str'):
    ncm.append(item.zfill(8))

df['NCM'] = ncm
    

lst_ISO = df['ISO'].to_list()
lst_cubagem = []
i = 0
for item in lst_ISO:
    
    if item[:2] == '45' or item[:2] == '42':
        lst_cubagem.append('60000')
    elif item[:2] == '22':
        lst_cubagem.append('30000')
       
    i = i +1
    
df['Cubagem'] = lst_cubagem


df_guarita = df[['ISO', 'Container', 'Tara', 'Peso Bruto', 'NCM', 'Lacre', 'Cubagem' ]]
guarita_dict = df_guarita.to_dict(orient = 'index')

In [ ]:
#sleep(3)
count = 0
#driver.maximize_window
frames = driver.find_elements()
driver.switch_to.default_content()
driver.switch_to.frame('Principal')

while count < len(guarita_dict):


    #for item in guarita_dict[count]:
        
    driver.switch_to.default_content()
    driver.switch_to.frame('Principal')
#ISO     

    # get xpath for ISO
    e = driver.find_element('name',"ItemCarga_tipoContainer")
    e = driver.find_element(By.XPATH, '/html/body/form/div/fieldset[3]/fieldset[1]/table/tbody/tr[1]/td[2]/input[1]')
    e.send_keys(guarita_dict[count]['ISO'])

    # print confirmação
    print(guarita_dict[count]['ISO'])
    sleep(0.5)

#CONTAINER

    # get xpath for Container 
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/fieldset[1]/table/tbody/tr[1]/td[4]/input')
    e.send_keys(guarita_dict[count]['Container'])

    # print confirmação
    print(guarita_dict[count]['Container'])
    sleep(0.5)

#TARA
    # get xpath for Tara
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/fieldset[1]/table/tbody/tr[2]/td[2]/input')
    e.send_keys(guarita_dict[count]['Tara'])

    # print confirmação
    print(guarita_dict[count]['Tara'])
    sleep(0.5)

#PESO_BRUTO

    peso = 0
    pesos = str(guarita_dict[count]['Peso Bruto']).replace(',','.').split('/')

    for item in pesos:
        peso += float(item)

    # get xpath for Peso Bruto
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/fieldset[1]/table/tbody/tr[2]/td[4]/input')
    e.send_keys("{:.0f}".format(peso * 1000)) # cálculo para ficar com o número certo de zeros. 

    # print confirmação
    print(guarita_dict[count]['Peso Bruto'])
    sleep(0.5)


#CUBAGEM

    # get xpath for CUBAGEM
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/fieldset[1]/table/tbody/tr[3]/td[2]/input')
    e.send_keys(guarita_dict[count]['Cubagem'])

    # print confirmação
    print(guarita_dict[count]['Cubagem'])
    sleep(0.5)

# NCM
    ncms = str(guarita_dict[count]['NCM']).replace(',','/').split('/')
    ncms = unique(ncms)

    # split "/" e loop for para múltiplas inserções
    for item in ncms:
        # get xpath for NCM
        e = driver.find_element(By.XPATH, '//*[@id="divCampos"]/fieldset[3]/table/tbody/tr/td[1]/fieldset/table/tbody/tr[1]/td[2]/input[1]')
        e.send_keys(item)
        # get xpath for "incluir"
        e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/table/tbody/tr/td[1]/fieldset/table/tbody/tr[2]/td/input')
        sleep(0.5)
        e.click()
        print(item)
        sleep(0.5)


    #get xpath for Lacre
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/table/tbody/tr/td[3]/fieldset/table/tbody/tr[1]/td[2]/input')
    e.send_keys(guarita_dict[count]['Lacre'])



    # get xpath for "incluir"
    e = driver.find_element('xpath', '//*[@id="divCampos"]/fieldset[3]/table/tbody/tr/td[3]/fieldset/table/tbody/tr[2]/td/input')
    sleep(0.5)
    e.click()
    print(guarita_dict[count]['Lacre'])
    sleep(0.5)

    #input('press enter to continue')


# INCLUIR ITEM        
    driver.find_element('xpath', '//*[@id="divCampos"]/p[2]/input[1]').click()

# INCLUIR NOVO ITEM - radio button

    driver.find_element('xpath', '/html/body/form[1]/fieldset[5]/table/tbody/tr/td[1]/input[1]').click()
    print('marcar inserir novo item')
    sleep(0.5)


# selecionar conteiner

    sel = Select(driver.find_element('xpath', '//*[@id="divItem"]/table/tbody/tr/td/select'))
    sel.select_by_value('1')

# ENVIAR     

    driver.find_element('xpath', '/html/body/form[1]/p[2]/input')
    print('clicar em enviar')
    sleep(0.5)

    count += 1
    
    if count >= len(guarita_dict):
        print('Processo Finalizado')

    else: 
        # input('Aperte enter para prosseguir')

        print('----- next one-----')

